In [1]:
# git config --global credential.helper store

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import itertools
from collections import Counter
import ssl
import time
import string
import unicodedata

from urllib.request import Request, urlopen
from threading import Thread
from bs4 import BeautifulSoup

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)

import nltk
nltk.download(["stopwords", "punkt", "averaged_perceptron_tagger", "maxent_treebank_pos_tagger"])
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk import RegexpParser
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

from clean_dfs import clean_features, clean_weeks, clean_lyrics
from web_scraping import parse_page, store_lyrics
from nlp_pipeline import lyrics_tokenize
from genre_helper_functions import get_bucket, contains_genre_type, create_genre_column
from make_plots import (make_frequency_plot, make_line_plot, make_dual_plot_same,
                        make_dual_plot_mixed, make_scatter)
import modeling_functions as mf

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]    

In [2]:
features = clean_features()
weeks = clean_weeks()

In [ ]:
'''
joined = weeks.merge(features, on='SongID')
#joined.to_csv("data/joined.csv", index=False)

# Expand genres into individual components
featureGenres = features.explode('spotify_genre')
featureGenres = featureGenres[featureGenres['spotify_genre'] != '']

joinedGenres = joined.explode('spotify_genre')
joinedGenres = joinedGenres[joinedGenres['spotify_genre'] != '']

explicitness = joined[['Year', 'spotify_track_explicit']]
explicitness = explicitness.groupby(['Year']).mean().reset_index()

numericalMetrics = joined.columns.tolist()[11:23]
numericals = joined[['Year'] + numericalMetrics].groupby(['Year']).mean().reset_index()
'''

In [ ]:
'''
# Normalize numerical features not between 0 and 1
featureGenresNorm = featureGenres.copy()
scaled = ["track_duration", "loudness", "tempo"]
for metric in scaled:
    mms = MinMaxScaler()
    featureGenresNorm[metric] = mms.fit_transform(featureGenresNorm['track_duration']. \
                                to_numpy().reshape(-1, 1))

# Create grouped tables
genres = featureGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoined = joinedGenres.groupby(['spotify_genre'])['SongID'].count().reset_index()
genresJoinedDecade = joinedGenres.groupby(['spotify_genre', 'Decade'])['SongID'].count(). \
                        reset_index().sort_values(by="Decade")
genreFeatures = featureGenresNorm.groupby(['spotify_genre'])[numericalMetrics].mean().reset_index()
'''

In [ ]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [24]:
testpage = parse_page("Dance the Night Away", "Twice")

In [ ]:
# Web scrape lyrics
featureScrape = features.loc[[contains_genre_type(genre, ["pop", "rock", "metal"]) for genre \
                             in features['spotify_genre']]].reset_index(drop=True)
lyricsMap = {}
threads = []
temp = 0
start = time.time()
# Write scraped lyrics to hashmap, parallelize to save time (thread safe because no unique keys)
#for i in range(temp, temp+50):
for i in range(len(featureScrape)):
    t = Thread(target=store_lyrics, args=(featureScrape['Song'][i],
                featureScrape['Performer'][i], lyricsMap))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
end = time.time()
print(end - start)
scrapedLyrics = pd.DataFrame(lyricsMap.items(), columns=["SongID", "Lyrics"])
scrapedLyrics.to_csv("data/scrapedLyrics.csv", index=False)

In [ ]:
# Get list of all improperly formatted songs and save to file
problemSongs = []
for k, v in lyricsMap.items():
    if v[0][0] == "*":
        problemSongs.append([k] + v[2:5])
print(len(featureScrape), len(problemSongs))

In [ ]:
with open("data/problemSongs.txt", "w") as file:
    for s in problemSongs:
        file.write("{}\n".format(s))

In [ ]:
###################

In [5]:
testpage = parse_page("Dance the Night Away", "Twice")
testpage = [line.replace(",", "") for line in testpage]
testpage = clean_lyrics(testpage)
print(testpage)
testpage_tokenized = lyrics_tokenize(testpage)
print(testpage_tokenized)
testpage2 = parse_page("Fake Love", "BTS")
testpage2 = [line.replace(",", "") for line in testpage2]
testpage2 = clean_lyrics(testpage2)
testpage2_tokenized = lyrics_tokenize(testpage2)
print(testpage2_tokenized)

Nanana nanana You and me in the moonlight A night of stars flowers and festivals The sound of the waves is on and we’re dancing right now This feels so perfect Hey ocean come play with us Hey wind come over here Under the moonlight it’s our world Everyone together party all night long yeah it’s good If you wanna have some fun Don’t let go of this special happiness That’s like the salty air One two three let’s go Let’s go dance like flying to space Hey! Let’s dance the night away Let’s dance the night away One two three let’s go Scream so loud they can hear you across the ocean Let’s dance the night away Dance the night away Let’s dance the night away Dance the night away Let’s dance the night away You and me in this cool night The half-moon is smiling Let’s promise to go behind the moon some day And throw a party yeah it’s good If you wanna have some fun Don’t let go of this special happiness That’s like the silver sand One two three let’s go Let’s go dance like flying to space Hey! Le

In [6]:
testcorpus = [testpage_tokenized, testpage2_tokenized]
print(testcorpus)
tfidf = TfidfVectorizer()
tfidfMatrix = tfidf.fit_transform(testcorpus)
print(tfidf.vocabulary_)
print(tfidfMatrix.todense()[0:10])

["b'nanana nanana a night flower sound feel hey hey moonlight world everyon parti night good wan dont special happi that salti air let let danc space hey let night let danc night let scream loud hear ocean let danc night danc night let danc night danc night let danc night night half-moon let promis moon day parti good wan dont special happi that silver let let danc space hey let night let danc night let scream loud hear ocean let danc night danc night let danc night danc night let danc night shout today last jump your higher higher shout today last starlight let danc night let danc night let scream loud hear ocean let danc night danc night let danc night danc night let danc night", 'b strong wish love perfect love weak cant dream cant true sick fake fake fake love sorri fake love fake fake love good man world everyth forest rout quit unsur mirror hell strong wish love perfect love weak cant dream cant true love bad love bad mold pretti lie love mad love mad tri eras doll love bad love 

In [ ]:
###################

In [2]:
# Read csv of previously outputted scraped lyrics and reformat to match original
allLyrics = clean_lyrics()

In [4]:
# NLP pipeline to create tokens from lyrics
allLyrics['Lyrics_tokenized'] = list(map(lyrics_tokenize, allLyrics['Lyrics']))
allLyrics.to_csv("data/allLyricsTokenized.csv", index=False)

KeyboardInterrupt: 

In [ ]:
# Create corpus and make TF-IDF
corpus = allLyrics['Lyrics_tokenized']
tfidf = TfidfVectorizer(max_features=200000)
tfidfMatrix = tfidf.fit_transform(corpus)
print(tfidf.vocabulary_)
print(tfidfMatrix.todense()[0:10])

In [22]:
allLyrics

,SongID,Lyrics,Lyrics_tokenized
0,The ArcherTaylor Swift,Combat I'm ready for combat I say I don't want...,b combat readi combat cruelti movi hundr throw...
1,TakeawayThe Chainsmokers & Illenium Featuring ...,Your heart for takeaway yeah yeah yeah-yeah Yo...,b'your heart takeaway yeah-yeah your heart tak...
2,Bad Bad BadYoung Thug Featuring Lil Baby,Wheezy outta here Rose gold seats on a fuckin'...,b wheezi outta rose gold seat helicopt doubl c...
3,HeartlessThe Weeknd,Young Metro young Metro young Metro (Sheesh) A...,b young metro young metro young metro sheesh a...
4,HeatChris Brown Featuring Gunna,Buddah bless this beat You likin' what you see...,b buddah bless beat everythin design that uh t...
5,Yellow HeartsAnt Saunders,She put my name with yellow hearts Her favorit...,b yellow heart favorit color star whenu2005sh ...
6,Don't Check On MeChris Brown Featuring Justin ...,If I'm out at the bar havin' a night on the to...,b'if i\ bar havin\ night town life parti pouri...
7,Cornelia StreetTaylor Swift,We were in the backseat Drunk on something str...,b'we backseat drunk someth stronger drink corn...
8,GoldenHarry Styles,Hey! Golden golden golden as I open my eyes Ho...,b hey golden golden golden open eye focus tou2...
9,Watermelon SugarHarry Styles,Tastes like strawberries on a summer evenin' A...,b tast strawberri summer evenin song moreu2005...


In [23]:
temp = pd.read_csv("data/scrapedLyrics.csv", converters={'Lyrics': lambda s: 
                            s[1:-1].split(", ")})

In [ ]:
model = Sequential()